In [2]:
import numpy as np
import pandas as pd
import nltk
import torch
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import pos_tag
from math import floor
from keras.preprocessing.text import Tokenizer
from bs4 import BeautifulSoup
import re
from keras.preprocessing.sequence import pad_sequences
# from torchvision.transforms import ToTensor
import torch.nn as nn

In [3]:
# data = pd.read_csv('Twitter_dataset.csv')
# del data['Flag']
# data.head()

In [4]:
# data = data.sample(frac=1).reset_index(drop=True)

# # data.head()

# data['target'][data['target']==4]=1

# data.head()

# data.to_csv('Twitter_dataset.csv', index=False)

Due to size of dataset, I was only able to upload 400000 rows of this dataset. Hence, in order to recreate this model, this minor adjustment is required.

In [5]:
data = pd.read_csv('Twitter_dataset.csv', nrows=500000)

data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

In [6]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [7]:
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)','',newString)
    newString = ''.join((contraction_mapping[t]+' ') if t in contraction_mapping else (t+' ') for t in newString.split(" "))
    newString = re.sub(r'"','',newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z\d]", " ", newString)
    tokens = ''.join((w+' ') if w not in stop_words else '' for w in newString.split())
    return newString

cleaned_text = []
max_words = 0
for i in data['Text']:
    max_words = max(max_words,len(i))
    cleaned_text.append(text_cleaner(i))
data['Cleaned Text'] = cleaned_text
print(max_words)


180


In [8]:
data.head()

,target,IDs,Date,User,Text,Cleaned Text
0,1,2185778769,Mon Jun 15 17:53:33 PDT 2009,TuckerPeterson,@JoeRinaldi They can do anything with CGI nowa...,joerinaldi they can do anything with cgi nowa...
1,1,2053725693,Sat Jun 06 05:30:43 PDT 2009,Curly_Wurlies,"@silentbobb oh totally, thats the plan yooh?",silentbobb oh totally thats the plan yooh
2,0,2251814388,Sat Jun 20 04:39:41 PDT 2009,rbmartin,@smokey_2009 I had to!,smokey 2009 i had to
3,0,2264180979,Sun Jun 21 03:02:31 PDT 2009,bwicaksono,I'm just testing my 1st augmented reality apps...,i am just testing my 1st augmented reality app...
4,0,2222352562,Thu Jun 18 06:51:47 PDT 2009,jesusowns9,I forgot my phone at the Vinskies' i feel so ...,i forgot my phone at the vinskies i feel so ...


In [9]:
X_train = np.array(data['Cleaned Text'])[:400000]
Y_train = np.array(data['target'])[:400000]
# Y_train = [[0,1] if w==0 else [1,0] for w in Y_train]
Y_train = torch.tensor(Y_train)
print(Y_train[0:5])

X_test = np.array(data['Cleaned Text'])[400000:]
Y_test = np.array(data['target'])[400000:]
# Y_test = [[0,1] if w==0 else [1,0] for w in Y_test]
Y_test = torch.tensor(Y_test)
print(Y_test.shape)

tensor([1, 1, 0, 0, 0])
torch.Size([96569])


In [10]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(X_train)

global x_train_tokens
x_train_tokens = x_tokenizer.texts_to_sequences(X_train)
x_test_tokens = x_tokenizer.texts_to_sequences(X_test)
x_train_tokens = pad_sequences(x_train_tokens,  maxlen=max_words, padding='post')
x_test_tokens = pad_sequences(x_test_tokens,  maxlen=max_words, padding='post')


x_train_tokens = torch.tensor(x_train_tokens)
x_test_tokens = torch.tensor(x_test_tokens)


In [11]:
print(x_train_tokens[4])
x_train_tokens = x_train_tokens.float()

tensor([    1,   472,     6,   214,    26,     3, 76844,     1,   109,    19,
         1416,   379,     8,     1,   251,    10,  1798,  1407,    42,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [50]:
batch_size = 100

class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.Sigmoid()
        self.h12 = nn.Linear(in_features=180, out_features=150)
        self.bn1 = nn.BatchNorm1d(num_features=150)
        self.h23 = nn.Linear(in_features=150, out_features=110)
        self.bn2 = nn.BatchNorm1d(num_features=110)
        self.h34 = nn.Linear(in_features=110, out_features=80)
        self.bn3 = nn.BatchNorm1d(num_features=80)
        self.h45 = nn.Linear(in_features=80, out_features=50)
        self.bn4 = nn.BatchNorm1d(num_features=50)
        self.h56 = nn.Linear(in_features=50, out_features=30)
        self.bn5 = nn.BatchNorm1d(num_features=30)
        self.h67 = nn.Linear(in_features=30,out_features=15)
        self.bn6 = nn.BatchNorm1d(num_features=15)
        self.h78 = nn.Linear(in_features=15,out_features=6)
        self.bn7 = nn.BatchNorm1d(num_features=6)
        self.h89 = nn.Linear(in_features=6,out_features=2)
        self.bn8 = nn.BatchNorm1d(num_features=2)
    def forward(self,x):
        x = self.h12(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.h23(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.h34(x)
        x = self.bn3(x)
        x = self.act(x)
        x = self.h45(x)
        x = self.bn4(x)
        x = self.act(x)
        x = self.h56(x)
        x = self.bn5(x)
        x = self.act(x)
        x = self.h67(x)
        x = self.bn6(x)
        x = self.act(x)
        x = self.h78(x)
        x = self.bn7(x)
        x = self.act(x)
        x = self.h89(x)
        x = self.bn8(x)
        x = self.act(x)
        return x
    
def train_OE(x_train_tokens,Y_train,loss_fn,optimizer):
    model.train()
    track_loss = 0
    correct = 0
    batches = int(len(x_train_tokens)/batch_size)
    for i in range(batches):
        training_data = x_train_tokens[batch_size*i:batch_size*(i+1)]
        labels = Y_train[batch_size*i:batch_size*(i+1)]
        pred = model(training_data)
        loss = loss_fn(pred,labels)
        track_loss += loss.item()
        correct += (torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            running_loss = round(track_loss/(i+1),2)
            running_acc = round((correct/((i+1)*batch_size))*100,2)
            print("Batch:", i+1, "/",batches, "Running Loss:",running_loss, "Running Accuracy:",running_acc)
        
    epoch_loss = track_loss/batches
    epoch_acc = (correct/len(x_train_tokens))*100
    return round(epoch_loss,2), round(epoch_acc,2)


def eval_OE(x_train_tokens,Y_train,loss_fn,optimizer):
    model.eval()
    track_loss = 0
    correct = 0
    batches = int(len(x_train_tokens)/batch_size)
    for i in range(batches):
        training_data = x_train_tokens[batch_size*i:batch_size*(i+1)]
        labels = Y_train[batch_size*i:batch_size*(i+1)]
        pred = model(training_data)
        loss = loss_fn(pred,labels)
        track_loss += loss.item()
        correct += (torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        

        if i%100==0:
            running_loss = round(track_loss/(i+1),2)
            running_acc = round((correct/((i+1)*batch_size))*100,2)
            print("Batch:", i+1, "/",batches, "Running Loss:",running_loss, "Running Accuracy:",running_acc)
        
    epoch_loss = track_loss/batches
    epoch_acc = (correct/len(x_train_tokens))*100
    return round(epoch_loss,2), round(epoch_acc,2)
    

model = ANN()
loss_fn=nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(params = model.parameters(),lr = lr)
epochs = 30

for i in range(epochs):
    print(f"Epoch-No: {i+1}")
    TE_loss, TE_acc = train_OE(x_train_tokens,Y_train,loss_fn,optimizer)
    VE_loss, VE_acc = eval_OE(x_train_tokens,Y_train,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", TE_loss, "Epoch Accuracy:", TE_acc)
    print("Inference:", "Epoch Loss:", VE_loss, "Epoch Accuracy:", VE_acc)
    print("--------------------------------------------------")




Epoch-No: 1
Batch: 1 / 4000 Running Loss: 0.7 Running Accuracy: 53.0
Batch: 101 / 4000 Running Loss: 0.69 Running Accuracy: 55.8
Batch: 201 / 4000 Running Loss: 0.69 Running Accuracy: 55.75
Batch: 301 / 4000 Running Loss: 0.69 Running Accuracy: 55.7
Batch: 401 / 4000 Running Loss: 0.68 Running Accuracy: 55.96
Batch: 501 / 4000 Running Loss: 0.68 Running Accuracy: 55.94
Batch: 601 / 4000 Running Loss: 0.68 Running Accuracy: 56.16
Batch: 701 / 4000 Running Loss: 0.68 Running Accuracy: 56.2
Batch: 801 / 4000 Running Loss: 0.68 Running Accuracy: 56.26
Batch: 901 / 4000 Running Loss: 0.68 Running Accuracy: 56.42
Batch: 1001 / 4000 Running Loss: 0.68 Running Accuracy: 56.44
Batch: 1101 / 4000 Running Loss: 0.68 Running Accuracy: 56.47
Batch: 1201 / 4000 Running Loss: 0.68 Running Accuracy: 56.46
Batch: 1301 / 4000 Running Loss: 0.68 Running Accuracy: 56.5
Batch: 1401 / 4000 Running Loss: 0.68 Running Accuracy: 56.58
Batch: 1501 / 4000 Running Loss: 0.68 Running Accuracy: 56.62
Batch: 1601 / 

KeyboardInterrupt: 

Accuracy:

Sigmoid: 57.2 %

ReLU: 55.7 %

BCELoss: 55.3 %

Achieved accuracy is not satisfactory. It is maybe because I used a simple ANN for this task. Hence, I am currently working towards doing sentiment analysis using LSTM